# 103089 - Data mining

<center><img src="M-UdL2.png"  width="300" alt="Italian Trulli"></center>

## Activity 1: Computing frequent pairs and association rules from transactions

Consider the following programming exercise. Given the information of the frequent singletons $(L_1)$ and frequent pairs $(L_2)$ we compute with our implementation of A-Priori for k=2 (at the notebook you have available), implement in spark functions to compute the confidence and interest of all the binary rules we can build from the set $L_2$. As the dataset to test your code, compute a set of transactions from this data set:

https://www.kaggle.com/datasets/heeraldedhia/groceries-dataset  (It is also included as an attachment to this assignment)

Observe that in this dataset each line is not a transaction, but a single purchased product in a given date by a given costumer. You must "aggregate" all the purchases by the same date and costumer to build a transaction.

You can also test your program with this smaller one (that it already contains one transaction per line) if you are not able to work with the previous one (but the maximum grade if only this dataset is used will be a 8):

https://www.kaggle.com/shazadudwadia/supermarket?select=GroceryStoreDataSet.csv

Try these values for $\theta$: 0.009, 0.01, 0.012

Number of frequent pairs you should get for each value of $\theta$ with the bigger data set  (Groceries_dataset.csv):

- $\theta$ = 0.009  ->  |L2| = 6
- $\theta$ = 0.01   ->  |L2| = 5
- $\theta$ = 0.012  ->  |L2| = 2

Once you have computed the sets T, L1, 1 and L2, your program should follow these steps:

Map, using mapPartitions, each frequent pair in the RDD with L2 to its list of binary association rules (two association rules per each different frequent pair). Use then the flattened version of this RDD.
Map each association rule of the previous resulting RDD, to a triple with (rule,confidence,interest). Observe that you will need to use the information in L1 and the number of transactions to compute these values. You can use the version of L1 stored as a python list in the driver (so it can be passed inside functions passed to spark tasks).
Finally, sort the association rules by their interest, and show back in the driver program the first 10 most interesting rules


You must present your program (together with its results after running it with the dataset I ask and with the three values of $\theta$) in a single python notebook.

### Cell 1: Initialization of PySpark

This cell initializes PySpark, a Python library for Spark, a big data processing framework. It sets up the environment, specifies the Python version to use, and creates a SparkContext (sc).

In [ ]:
import pyspark
import os

os.environ["PYSPARK_PYTHON3"] = "python"
spark_home = os.environ.get('SPARK_HOME', None)
print (spark_home)
sc = pyspark.SparkContext.getOrCreate()
sc.setLogLevel('ERROR')
sc

### Cell 2: Loading Data
This cell loads a dataset named Groceries_dataset.csv into a pandas DataFrame (df). It displays the first few rows of the DataFrame to give an overview of the data

In [ ]:
import pandas as pd
df = pd.read_csv("./Groceries_dataset.csv")
df.head()

### Cell 3: Data Cleaning and Grouping
We processes a Spark DataFrame by removing duplicates, grouping the data by 'Member_number' and 'Date' while aggregating other columns with comma-separated values, and then further manipulating the 'itemDescription' column to have a cleaner format.

In [ ]:
df=df.drop_duplicates()
grouped =  df.groupby(['Member_number', 'Date']).agg(lambda x: ','.join(x)).reset_index()
grouped['itemDescription'] = grouped['itemDescription'].str.split(',')
grouped['itemDescription'] = grouped['itemDescription'].astype(str).str.replace('[', '').str.replace(']', '').str.replace('\'', '')
grouped.head()

### Cell 4: Creating a List of Items
Now we extract the 'itemDescription' column from the grouped DataFrame, which has been previously processed, and then applies the str.split(', ') method to split the comma-separated strings into lists of items. These lists are then converted to a nested list using .tolist().

In [ ]:
items_list = grouped['itemDescription'].str.split(', ').tolist()
print(items_list)

### Cell 5: Creating an RDD in Spark
rddT.collect() will return a list of lists, where each inner list contains the items associated with a specific combination of 'Member_number' and 'Date' that we previously split and prepared. The data is now available in the driver program for further analysis or processing.

In [ ]:
session = pyspark.sql.SparkSession(sc)
rddT =  sc.parallelize(items_list)
rddT.collect()

### Cell 6: Compute $L_1$ and $T_{L_1}$

In [ ]:
# Compute the rdd with frequent singleton sets (L_1)
def computeL1 ( rddT, numtrans, theta ):
  rddtemp = rddT.flatMap( lambda t : [ (it,1) for it in t ] ).reduceByKey( lambda a,b : a+b  )
  return rddtemp.filter( lambda x : (float(x[1])/numtrans) >= theta )

# Map any transaction to its version without elements not in L1
# L1 must be a python list, not a RDD
def computeTfilteredByL1( seqOfT, L1 ):
    for t in seqOfT:
       yield [ it for it in t if (it in L1) ]

### Cell 7: Compute $L_2$ from $C_2(T)$

In [ ]:
# For each t in seqofFilteredT (they come from T_{L_1}), compute pairs (a,b) from t that belong to C_2
def generateC2( seqofFilteredT ):
    for t in seqofFilteredT:
      cpairslist = []
      for (a,b) in [ (a,b) for i,a in enumerate(t[:-1]) for b in t[i+1:] ]:
                cpairslist.append( ((a,b),1) if (a <= b) else ((b,a),1)  )         
      yield cpairslist

### Cell 8: Compute $L_2$ from $C_2(T)$

In [ ]:
def computeL2( rddC2T, numtrans, theta ):
    pairsCountedrdd = rddC2T.reduceByKey( lambda v1,v2 : v1+v2 )
    # Finally, filter out from the previous rdd those pairs with frequency below theta
    return pairsCountedrdd.filter( lambda x : (float(x[1])/numtrans) >= theta )

### Cell 9: theta values

In [ ]:
numtrans = len(items_list)
theta = 0.009 # ->  |L2| = 6
#theta = 0.01 #   ->  |L2| = 5
#theta = 0.012 # ->  |L2| = 2

### Cell 10: A-Priori algorithm (Phase 1)
Compute the RDD for $L_1$ and $T_{L_1}$ and convert it to a python list back to the driver (remember, this may not scale well depending on the size of $L_1$):

In [ ]:
rddL1 = computeL1(rddT, numtrans, theta)
L1 = rddL1.keys().collect() # we need only the items (keys) in final L1
TL1 = rddT.mapPartitions( lambda seqOfT : computeTfilteredByL1( seqOfT, L1 )  )

### Cell 11: A-Priori algorithm (Phase 2)
Compute $C_2(T)$ from $T_{L_1}$

In [ ]:
rddC2T = TL1.mapPartitions( lambda seqOfFilteredT : generateC2( seqOfFilteredT ) )
rddC2TFlat = rddC2T.flatMap( lambda x : x )

### Cell 12: A-Priori algorithm (Phase 3)
Compute 𝐿2 from 𝐶2(𝑇)

In [ ]:
rddL2 = computeL2( rddC2TFlat, numtrans, theta )
rddL2 = rddL2.sortBy(lambda a: -a[1])
for it in rddL2.toLocalIterator():
    print (it)

print(rddL2.count())

### Cell 13: Finding association rules

In [ ]:
def generate_association_rules(rddL2):
    for rule in rddL2:
        yield [((rule[0][0], rule[0][1]), rule[1])]
        yield [((rule[0][1], rule[0][0]), rule[1])]

rules_rdd = rddL2.mapPartitions(generate_association_rules)
flatt_rdd = rules_rdd.flatMap(lambda x: list(x))
results = flatt_rdd.collect()

print(results)

### Cell 14: Confidence

In [ ]:
def confidence(support_A,support_B):
    return support_A[1] / support_B[1]

#confidence(rddL1)

### Cell 15: Interest

In [ ]:
def interest(support_AB,support_A,support_B,long):
    return confidence(support_AB,support_A) - (support_B[1] / long)

### Cell 16: TripleRDD

In [ ]:
def tripleRDD(rules,rdd,len):
    for item in rdd:
        if item[0] == rules[0][0]: x = item[1]
        if item[0] == rules[0][1]: y = item[1]

    return (rules[0],             # Rule
            rules[1] / x,         # Confidence
            rules[1] / x - y / len) # Interest

### Cell 17: Results

In [ ]:
rddL1list = rddL1.collect()
interestConfidence = flatt_rdd.map(lambda rule: tripleRDD(rule, rddL1list,numtrans))
sorted_data = interestConfidence.sortBy(lambda x: x[2], ascending=False)
first_10_values = sorted_data.take(10)
#sorted_data = sorted(first_10_values, key=lambda x: x[2], reverse=True)
for value in first_10_values:
    print(value)

print(interestConfidence.count())

##

- $\theta$ = 0.009  ->  |L2| = 6

(('whole milk', 'yogurt'), 0.07067287346593314, -0.015205626834808694)  
(('other vegetables', 'soda'), 0.07936507936507936, -0.01774111591661548)  
(('whole milk', 'rolls/buns'), 0.08844688954718578, -0.021557788659056276)  
(('soda', 'other vegetables'), 0.09979353062629043, -0.022307652291573637)  
(('whole milk', 'soda'), 0.07363520947947524, -0.023470985802219596)  
(('other vegetables', 'rolls/buns'), 0.08648056923918993, -0.023524108967052135)  
(('rolls/buns', 'other vegetables'), 0.0959902794653706, -0.026110903452493464)  
(('yogurt', 'whole milk'), 0.12996108949416343, -0.027961786934360244)  
(('whole milk', 'other vegetables'), 0.09394837071519255, -0.028152812202671518)  
(('rolls/buns', 'whole milk'), 0.12697448359659783, -0.030948392831925853)  


- $\theta$ = 0.01   ->  |L2| = 5

(('whole milk', 'yogurt'), 0.07067287346593314, -0.015205626834808694)  
(('whole milk', 'rolls/buns'), 0.08844688954718578, -0.021557788659056276)  
(('whole milk', 'soda'), 0.07363520947947524, -0.023470985802219596)  
(('other vegetables', 'rolls/buns'), 0.08648056923918993, -0.023524108967052135)  
(('rolls/buns', 'other vegetables'), 0.0959902794653706, -0.026110903452493464)  
(('yogurt', 'whole milk'), 0.12996108949416343, -0.027961786934360244)  
(('whole milk', 'other vegetables'), 0.09394837071519255, -0.028152812202671518)  
(('rolls/buns', 'whole milk'), 0.12697448359659783, -0.030948392831925853)  
(('other vegetables', 'whole milk'), 0.12151067323481117, -0.03641220319371251)  
(('soda', 'whole milk'), 0.11975223675154852, -0.03817063967697516)  

- $\theta$ = 0.012  ->  |L2| = 2

(('whole milk', 'rolls/buns'), 0.08844688954718578, -0.021557788659056276)  
(('whole milk', 'other vegetables'), 0.09394837071519255, -0.028152812202671518)  
(('rolls/buns', 'whole milk'), 0.12697448359659783, -0.030948392831925853)  
(('other vegetables', 'whole milk'), 0.12151067323481117, -0.03641220319371251)  

##